In [2]:
import pandas as pd
import numpy as np
import os
import torch
import os
import sys
import json
import time
import requests
import datetime
import dateutil
from dateutil.relativedelta import relativedelta
import xml.etree.ElementTree as et 
import tarfile
from pathlib import Path

In [3]:
nyt_api_key = '6iiiMFc1qUHG4P4GtEvsZwv5oaAvCbrn'
identity_terms = [
    'muslim',
    'jew',
    'jews',
    'white',
    'islam',
    'blacks',
    'muslims',
    'women',
    'whites',
    'gay',
    'black',
    'democrat',
    'islamic',
    'allah',
    'jewish',
    'lesbian',
    'transgender',
    'race',
    'brown',
    'woman',
    'mexican',
    'religion',
    'homosexual',
    'homosexuality',
    'africans'
]

In [8]:

start_date_str = "1987-01-01 00:00:00.0"
end_date_str = "2007-06-19 00:00:00.0"
# start = datetime.date.strftime(start_date_str, '%Y-%m-%d %H:%M:%S.%f')
# end = datetime.date.strftime(end_date_str, '%Y-%m-%d %H:%M:%S.%f')
# months_in_range = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %-m").tolist()]

api_key = "6iiiMFc1qUHG4P4GtEvsZwv5oaAvCbrn"

In [12]:
def untargzirator(fname, new_location):
    Path(new_location).mkdir(parents=True, exist_ok=True)
    if fname.endswith(".tgz"):
        tar = tarfile.open(fname, "r")
        tar.extractall(new_location)
        tar.close()
    elif fname.endswith("tar"):
        tar = tarfile.open(fname, "r:")
        tar.extractall(new_location)
        tar.close()

In [11]:
extracted_files_path = '../data/nyt_corpus/extracted_data/'
extracted_files_path_tar = '../data/nyt_corpus/tar_data/'
year_files = os.listdir('../data/nyt_corpus/data')
# for year in year_files[0]:
year = '1987'
months_zipped = os.listdir(f'../data/nyt_corpus/data/{year}/')
#print(months_zipped)
new_location = extracted_files_path + year
#for month in months_zipped[0]:
month = '01.tgz'
untargzirator(f'../data/nyt_corpus/data/{year}/{month}', new_location)
print(new_location)            


ReadError: invalid header

In [ ]:
# code adapted from: https://towardsdatascience.com/collecting-data-from-the-new-york-times-over-any-period-of-time-3e365504004
def send_request(date, query):
    '''Sends a request to the NYT Archive API for given date.'''
    base_url = 'https://api.nytimes.com/svc/search/v2/'
    url = base_url + query + date[0] + '/' + date[1] + '.json?api-key=' + api_key
    response = requests.get(url).json()
    time.sleep(6)
    return response

def is_valid(article, date):
    '''An article is only worth checking if it is in range, and has a headline.'''
    is_in_range = date > start and date < end
    has_headline = type(article['headline']) == dict and 'main' in article['headline'].keys()
    return is_in_range and has_headline

def parse_response(response):
    '''Parses and returns response as pandas data frame.'''
    data = {'headline': [],  
        'date': [], 
        'doc_type': [],
        'material_type': [],
        'section': [],
        'keywords': []}
    
    articles = response['response']['docs'] 
    for article in articles: # For each article, make sure it falls within our date range
        date = dateutil.parser.parse(article['pub_date']).date()
        if is_valid(article, date):
            data['date'].append(date)
            data['headline'].append(article['headline']['main']) 
            if 'section' in article:
                data['section'].append(article['section_name'])
            else:
                data['section'].append(None)
            data['doc_type'].append(article['document_type'])
            if 'type_of_material' in article: 
                data['material_type'].append(article['type_of_material'])
            else:
                data['material_type'].append(None)
            keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            data['keywords'].append(keywords)
    return pd.DataFrame(data) 


def get_data(dates):
    '''Sends and parses request/response to/from NYT Archive API for given dates.'''
    total = 0
    print('Date range: ' + str(dates[0]) + ' to ' + str(dates[-1]))
    if not os.path.exists('headlines'):
        os.mkdir('headlines')
    for date in dates:
        response = send_request(date)
        df = parse_response(response)
        total += len(df)
        df.to_csv('../data/nyt/headlines/' + date[0] + '-' + date[1] + '.csv', index=False)
        print('Saving headlines/' + date[0] + '-' + date[1] + '.csv...')


query = "body.search=election"
querystring = {f'articlesearch.json?q={query}&api-key={nyt_api_key}'}


In [4]:
response = requests.request("GET", url, params=querystring)

ValueError: too many values to unpack (expected 2)

In [ ]:
# https://catalog.ldc.upenn.edu/LDC2008T19
# from January 1, 1987 and June 19, 2007